# Database Helper
Database connection and query utilities.
Run this with: `%run /workspace/system/helpers/01_Database.ipynb`

In [ ]:
# Load configuration if not already loaded
if 'DB_CONFIG' not in globals():
    %run /workspace/system/helpers/00_Config.ipynb


import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from typing import Any, Optional, Dict

In [ ]:
def get_connection():
    """Create and return a PostgreSQL database connection."""
    return psycopg2.connect(
        host=DB_CONFIG['server'],
        port=DB_CONFIG['port'],
        database=DB_CONFIG['database'],
        user=DB_CONFIG['username'],
        password=DB_CONFIG['password']
    )

def execute_query(query: str, params: tuple = None) -> pd.DataFrame:
    """Execute a SELECT query and return results as DataFrame."""
    with get_connection() as conn:
        # Use RealDictCursor to get column names
        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
            cursor.execute(query, params)
            results = cursor.fetchall()
            if results:
                return pd.DataFrame(results)
            return pd.DataFrame()

def execute_command(query: str, params: tuple = None) -> int:
    """Execute an INSERT/UPDATE/DELETE command and return rows affected."""
    with get_connection() as conn:
        with conn.cursor() as cursor:
            cursor.execute(query, params)
            conn.commit()
            return cursor.rowcount

def execute_scalar(query: str, params: tuple = None) -> Any:
    """Execute a query and return a single value."""
    with get_connection() as conn:
        with conn.cursor() as cursor:
            cursor.execute(query, params)
            result = cursor.fetchone()
            return result[0] if result else None

def test_connection() -> bool:
    """Test database connection."""
    try:
        result = execute_scalar("SELECT 1")
        return result == 1
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return False

def get_active_cycle() -> Optional[Dict[str, Any]]:
    """Get the active cycle information."""
    query = """
    SELECT c.*, l.locked_by, l.locked_at
    FROM irp_cycle c
    INNER JOIN irp_system_lock l ON l.active_cycle_id = c.id
    WHERE c.status = 'active'
    """
    df = execute_query(query)
    if not df.empty:
        return df.iloc[0].to_dict()
    return None

def init_database() -> bool:
    """Initialize database from SQL script."""
    sql_file = SYSTEM_PATH / 'db' / 'init_database.sql'
    if not sql_file.exists():
        print(f"❌ Database init script not found: {sql_file}")
        return False
    
    try:
        with get_connection() as conn:
            with conn.cursor() as cursor:
                # Read and execute the SQL script
                with open(sql_file, 'r') as f:
                    sql_script = f.read()
                
                # PostgreSQL can execute the entire script at once
                cursor.execute(sql_script)
                conn.commit()
                
                print("✅ Database initialized successfully")
                return True
    except Exception as e:
        print(f"❌ Database initialization failed: {e}")
        return False

# Test connection on load
if test_connection():
    print("✅ Database connection successful")
else:
    print("⚠️  Database connection failed - you may need to initialize the database")